In [26]:
import pandas as pd
import numpy as np
from skimage.measure import label, regionprops
import trackpy as tp
import matplotlib as mpl
import matplotlib.pyplot as plt
from os.path import isfile, join
import datetime
from tifffile import imread
import numba as nb

In [17]:
'''
batch process the segmented tiles of the image and just get the data
'''

crox = True             #choose whether to batch process light particles or dark particles
dark_regions = False
movie = 3
movie = str(movie)
minparticlesize = 5
maxparticlesize = 200
search_range = 5        # how many pixels the particles are allowed to jump between frames- 15 for dark regions and 3 for bright crox particles
mintracklength = 15     #filtering by track lengths of 15 frames works well for the dark regions, but for the light particles a smaller number is better
mypath = 'median_8000_masked.tif'

In [24]:

def findparticles_3d_img(frames: np.array, minsize=5, maxsize = 200):
    """
        This function takes a 3d image "frames" with the structure [frame, x, y]

       Create a pandas dataframe, and then find the particles (regions) using skimage's label function.  Label connects
       regions of the same integer value, i.e. segmented regions. In this dataframe, I also save the perimeter, filled fraction,
       and the area.
    """
    black = np.min(frames)
    white = np.max(frames)

    features = pd.DataFrame()
    for num, img in enumerate(frames):
        label_image, number_of_labels = label(img, background=black, return_num=True)

        for region in regionprops(label_image, intensity_image=img):
            # Everywhere, skip small areas
            if region.area < minsize:
                continue
            elif region.area > maxsize:
                continue
            # Store features which survived to the criterions
            features = features.append([{'y': region.centroid[0],
                                         'x': region.centroid[1],
                                         'area': region.area,
                                         'perimeter': region.perimeter,
                                         'fraction filled': region.solidity,
                                         'frame': num,
                                         }])
            
        print(num)

    return features

In [19]:
segmented = imread(mypath)
features = findparticles_3d_img(segmented, minparticlesize)     #minsize of 5 for the crox particles, and 15 for the dark regions
features.to_csv("median_8000_masked_notrackpy.csv")

Frame 268: 192 trajectories present.


In [ ]:
master = pd.DataFrame()
t = tp.link_df(features, search_range, memory=25)
t1 = tp.filter_stubs(t, mintracklength)      #filtering by track lengths of 15 frames works well for the dark regions, but for the light particles a smaller number is better

master = master.append(t1)


In [20]:
master.to_csv('median_8000_masked.csv')